In [1]:
import pandas as pd
import sqlite3
# functiont to include some html in the cell for visualization purposes
from IPython.display import HTML


In [2]:
# Create connection to SQLite
conn = sqlite3.connect('Data/chinook-unmodified.db')

In [3]:
# I define an helper function to run SQL query more quickly
def run_query(query):
    output = pd.read_sql_query(query, conn)
    return output

### Database Schema

In [4]:
HTML('<img src="chinook-schema.svg" width=500>')

##### 1. Which music genres are most popular among U.S. customers, and what share of total U.S. track sales does each represent?


The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|Artist Name|          Genre|
|-----------|---------------|
|Regal	    |    Hip-Hop    |
|Red Tone	|     Punk      | 
|Meteor and the Girls| Pop  |
|Slim Jim Bites|	Blues   |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

You'll need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store.

In [5]:
q1 = '''
SELECT 
	g.name AS genre,
	COUNT(*) AS tracks_sold,
	-- multiply by 1.0 to force FLOAT division by converting COUNT(*) to a FLOAT type
	ROUND( COUNT(*) * 100.0/ SUM(COUNT(*)) OVER (), 2) || " %" AS share_of_US_tracks
FROM customer AS c
JOIN invoice AS i USING(customer_id)
JOIN invoice_line AS il USING(invoice_id)
JOIN track AS t USING(track_id)
JOIN genre AS g USING(genre_id)
WHERE lower(i.billing_country) = "usa"
GROUP BY g.name
HAVING g.name IN ('Hip Hop/Rap', 'Alternative & Punk', 'Pop', 'Blues')
ORDER BY COUNT(*) DESC;
'''

run_query(q1)


,genre,tracks_sold,share_of_US_tracks
0,Alternative & Punk,130,62.5 %
1,Blues,36,17.31 %
2,Pop,22,10.58 %
3,Hip Hop/Rap,20,9.62 %


#### 📊 Findings and Recommendation

This initial analysis reveals that **Alternative & Punk**, **Blues**, and **Pop** are the top-performing genres among the four candidates, based on the number of tracks purchased by U.S. customers.

By filtering the results to only the genres associated with the new artists under consideration, we find that:
- Alternative & Punk accounts for 62.5% of relevant track sales
- Blues follows with 17.31%
- Pop holds 10.58% of the share

Although Rock and Metal have higher overall sales in the full dataset, they are not associated with the candidate artists.

Given the U.S-focused marketing strategy and available data, the recommended artists to prioritize are:
1. Red Tone (Alternative & Punk)
2. Slim Jim Bites (Blues)
3. Meteor and the Girls (Pop)

This selection is based on historical U.S. customer preferences and should align well with the store’s strategic goals.

#### 2. How much revenue has each sales support agent generated, and what customer or regional factors might explain their performance?

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

You might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.

In [10]:
q2 = ''' 
SELECT 
	e.employee_id, 
	e.last_name,
	-- deal with null values if some reps generated no sales
	COALESCE(SUM(i.total), 0) AS total_usd_generated_sales
FROM employee AS e
-- I prefer a LEFT JOIN to show if any employee has served no customers
LEFT JOIN customer AS c 
ON e.employee_id = c.support_rep_id
LEFT JOIN invoice AS i USING(customer_id)
GROUP BY e.employee_id, e.last_name
ORDER BY SUM(i.total) DESC;
'''

run_query(q2)

,employee_id,last_name,total_usd_generated_sales
0,3,Peacock,1731.51
1,4,Park,1584.00
2,5,Johnson,1393.92
3,1,Adams,0.00
4,2,Edwards,0.00
5,6,Mitchell,0.00
6,7,King,0.00
7,8,Callahan,0.00


In [11]:
q2_1 = ''' 
-- Explore potential factors
WITH generated_sales AS (
	SELECT 
			e.employee_id, 
			e.last_name,
			-- deal with null values if some reps had assisted no customers and generated no sales
			COALESCE( SUM(i.total), 0) AS total_usd_generated_sales
	FROM employee AS e
	-- I prefer a LEFT JOIN to show if any employee has served no customers
	LEFT JOIN customer AS c 
	ON e.employee_id = c.support_rep_id
	LEFT JOIN invoice AS i USING(customer_id)
	GROUP BY e.employee_id, e.last_name
	ORDER BY SUM(i.total) DESC
)
SELECT
	gs.last_name,
	gs.total_usd_generated_sales,
	-- investigate hire duration
    -- I use JULIANDATE that is specific to sqlite to calculate the difference in days then convert to years
	ROUND((JULIANDAY(DATE('now')) - JULIANDAY(DATE(e.hire_date)))/365, 2) AS years_of_employment,
	-- investigate role
	e.title AS role,
	e.state,
	e.city
FROM generated_sales AS gs
LEFT JOIN employee AS e USING(employee_id);
'''

run_query(q2_1)

,last_name,total_usd_generated_sales,years_of_employment,role,state,city
0,Peacock,1731.51,8.17,Sales Support Agent,AB,Calgary
1,Park,1584.00,8.08,Sales Support Agent,AB,Calgary
2,Johnson,1393.92,7.62,Sales Support Agent,AB,Calgary
3,Adams,0.00,8.80,General Manager,AB,Edmonton
4,Edwards,0.00,9.08,Sales Manager,AB,Calgary
5,Mitchell,0.00,8.62,IT Manager,AB,Calgary
6,King,0.00,8.41,IT Staff,AB,Lethbridge
7,Callahan,0.00,8.24,IT Staff,AB,Lethbridge


#### 📊 Findings and Recommendationfter 

This analysis highlights that only three employees (Peacock, Park, and Johnson) are responsible for all recorded sales, generating a combined total revenue of $4,709.43 USD. The remaining employees show zero recorded sales.

To investigate potential reasons behind this performance gap, we analyzed:
- **Years of Employment**: There’s no consistent pattern between tenure and sales performance. For example, Park and Peacock both have similar years of experience, yet Peacock leads in revenue.  

- **Job Titles**: Only the three sales-performing employees have the title “Sales Support Agent”, suggesting the others likely serve in non-sales roles (e.g., IT staff or management).  

- **Geography** (State and City): All revenue-generating employees are based in Canada, which aligns with the regions their assigned customers may belong to.  

#### ✅ Conclusion:

The differences in performance are not due to behavioral reasons. Only three employees are assigned customer-facing sales duties.  
To evaluate sales performance fairly it is suggested to:  

- Narrow the analysis to employees with the role Sales Support Agent.
- Optionally, exploore more about customer distribution by region to ensure a balanced workload across agents.

#### 3. Which countries represent our most valuable customer markets, and how should we group low-representation countries for reporting?

Your next task is to analyze the sales data for customers from each different country.  

In particular, you have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value  

Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis.


In [12]:
q3 = ''' 
-- What do I need to answer this question?
-- 1. list of customer -> customer table
-- 2. country of each customer -> customer table
-- 3. number of customer by each country -> customer table
-- 3. list of orders (for total sales value) -> invoice table
-- 4. average sales per country -> invoice table 

WITH data_by_country AS (
	SELECT
		c.country,
		COUNT(DISTINCT customer_id) AS num_of_customers,
		SUM(i.total) AS total_sales,
		SUM(i.total)/COUNT(DISTINCT customer_id) AS avg_sales_per_customer,
		AVG(i.total) AS avg_sales_per_order
	FROM customer AS c
	JOIN invoice AS i USING(customer_id)
	GROUP BY c.country
	ORDER BY c.country
),
country_labeled AS (
	SELECT
		CASE WHEN
			num_of_customers = 1 THEN 'Other' ELSE country END AS country,
		num_of_customers,
		total_sales,
		avg_sales_per_customer,
		avg_sales_per_order
	FROM data_by_country
)
SELECT 
	country,
	SUM(num_of_customers) AS number_of_customers,
	SUM(total_sales) AS total_sales,
	-- Basically here I just calculate the average values for the 'Other' countries.
    -- I avoid round to avoid losing precision when I am exporting the data to Power Bi.
	AVG(avg_sales_per_customer) AS avg_sales_per_customer,
	AVG(avg_sales_per_order) AS avg_sales_per_order
FROM country_labeled
GROUP BY country
ORDER BY
-- Put 'Other' countries last and then order by total sales
	CASE WHEN
		country='Other' THEN 0
	ELSE 1 END DESC,
	total_sales DESC;
'''
run_query(q3)

,country,number_of_customers,total_sales,avg_sales_per_customer,avg_sales_per_order
0,USA,13,1040.49,80.037692,7.942672
1,Canada,8,535.59,66.948750,7.047237
2,Brazil,5,427.68,85.536000,7.011148
3,France,5,389.07,77.814000,7.781400
4,Germany,4,334.62,83.655000,8.161463
5,Czech Republic,2,273.24,136.620000,9.108000
6,United Kingdom,3,245.52,81.840000,8.768571
7,Portugal,2,185.13,92.565000,6.383793
8,India,2,183.15,91.575000,8.721429
9,Other,15,1094.94,72.996000,7.445071


#### 📊 Findings and Recommendation

The United States is clearly the most valuable customer market, producing the highest total sales and representing the largest customer base (13 customers with over $1,040 in total sales). Canada, Brazil, and France also show strong performance across both customer count and total revenue.

To simpiply the result, countries with only one customer were grouped under the label “Other”. While this group collectively contributed the highest total sales after the USA, it’s composed of many low-representation markets, which makes individual country-level insights less actionable.

Interestingly, countries like Portugal, India, and Czech Republic, despite having just 2 customers each, show high average sales per customer. This suggests potential for an expansion or personalized engagement strategies in these smaller yet high-value markets.

#### 4. Among invoices that contain tracks from a single album, what proportion represent full album purchases versus partial (individual track) purchases?

The Chinook music store allows customers to either purchase a full album or select individual tracks. Management is considering a cost-saving strategy:   
licensing only popular tracks instead of full albums.

To make an informed decision, they want to know how often customers are actually buying entire albums rather than just hand-picking tracks.  

##### 📝 Analytical Assumptions 

This analysis excludes invoices with tracks from multiple albums. Although they exist, we focus only on invoices with one album to produce reliable, interpretable results. In this filtered group:
- If the invoice contains all tracks from the album → it’s a Full Album Purchase
- If it contains only some tracks → it’s a Mixed Purchase


In [9]:
q4 = '''
WITH invoices_data AS (
    -- I find invoices with songs from only one album and their respective number of songs
    SELECT
        il.invoice_id,
        -- use a correlated subquery to get album_id of each invoice that has one album
        (SELECT DISTINCT album_id 
         FROM invoice_line 
         JOIN track USING(track_id)
         WHERE il.invoice_id = invoice_line.invoice_id) AS album_id,
        -- count how many tracks are in the invoice, which, given our filter in the having clause,
        -- will return the number of tracks in the purchased album 
        COUNT(t.track_id) AS tracks_count
    FROM invoice_line AS il
    JOIN track AS t USING(track_id)
    GROUP BY il.invoice_id
    -- if an invoice has only one album that might be a possible candidate to identify it as "album purchase"
    -- automatically exclude all the invoices with more than one album
    HAVING COUNT(DISTINCT t.album_id) = 1
),
inventory_data AS (
    -- I find how many songs each album has in the chinook inventory
    SELECT
        a.album_id,
        COUNT(t.track_id) AS tracks_count
    FROM album AS a
    JOIN track AS t USING(album_id)
    GROUP BY a.album_id
),
invoice_category AS (
    -- Categorize each invoice
    SELECT 
        invoices_data.invoice_id,
        inventory_data.album_id,
        CASE
            WHEN invoices_data.tracks_count = inventory_data.tracks_count THEN 'Album Purchase'
            WHEN invoices_data.tracks_count <> inventory_data.tracks_count THEN 'Mixed Purchase'
            ELSE 'Error' END AS purchase_type
    FROM invoices_data
    JOIN inventory_data USING(album_id)
)
-- Calculate the percentage and frequency of each invoice type
SELECT
    purchase_type,
    COUNT(*) AS num_of_invoices,
    -- Subquery to select the total num of invoices
    100.00 * COUNT(*) / (SELECT COUNT(*) FROM invoice) AS percentage
FROM invoice_category
GROUP BY purchase_type;
'''

run_query(q4)

,purchase_type,num_of_invoices,percentage
0,Album Purchase,114,18.566775
1,Mixed Purchase,57,9.283388


#### 📊 Findings and Recommendation

Out of all invoices in the dataset, only a small subset (171 invoices) contain tracks from a single album. Within this filtered group:  

- 114 invoices (67%) represent full album purchases (i.e., the invoice contains all tracks from a single album).
- 57 invoices (33%) represent partial purchases, where the customer selected only some tracks from the album.  
  
  
When considering all invoices in the database, this translates to:  

- 18.57% of total invoices are full album purchases.
- 9.28% are partial album purchases from a single album.

These results suggest that while full album purchases do occur with some regularity, a meaningful portion of customers still prefer to buy individual tracks, even when they’re purchasing from just one album. This supports management’s idea to move toward selling popular tracks only, as long as the store continues to accommodate full-album buyers.
